# Recommender System

## Import Libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats
import graphlab as gl
import dill
#dill.load_session('notebook_env.db')

## Upload Song Data

In [2]:
songs_path = ('/Users/ivettetapia 1/Symbolic Link Seagate Drive/Springboard/Capstone 1_Mus_Recomend/Data/msd_song_metadata.csv')

songs = pd.read_csv(songs_path, na_values = 'NaN', index_col = 0)

<font color='green'> **Drop columns that are not relevant for recomendation below. These are either columns with 0 values (danceability and energy), columns that have ids/indexes or represent confidence in song measurements.**</font>

In [3]:
columns = ['artist_7digitalid','artist_mbid','artist_playmeid','idx_artist_terms',
           'idx_similar_artists','release_7digitalid','track_7digitalid','analysis_sample_rate',
           'audio_md5','danceability','energy','idx_bars_confidence',
           'idx_bars_start','idx_beats_confidence','idx_sections_start',
           'idx_segments_confidence','idx_segments_loudness_max','idx_segments_loudness_max_time',
           'idx_segments_loudness_start','idx_segments_pitches','idx_segments_start','idx_segments_timbre',
           'idx_tatums_confidence','idx_tatums_start','key_confidence','mode_confidence',
           'time_signature_confidence','track_id','idx_artist_mbtags','idx_beats_start','idx_sections_confidence']

songs = songs.drop(labels = columns, axis = 1)

print(songs.info())

songs.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 905712 entries, 0 to 981021
Data columns (total 20 columns):
artist_familiarity    905531 non-null float64
artist_hotttnesss     905700 non-null float64
artist_id             905712 non-null object
artist_latitude       325985 non-null float64
artist_location       460185 non-null object
artist_longitude      325985 non-null float64
artist_name           905712 non-null object
release               905712 non-null object
song_hotttnesss       551532 non-null float64
song_id               905712 non-null object
title                 905708 non-null object
duration              905712 non-null float64
end_of_fade_in        905712 non-null float64
key                   905712 non-null int64
loudness              905712 non-null float64
mode                  905712 non-null int64
start_of_fade_out     905712 non-null float64
tempo                 905712 non-null float64
time_signature        905712 non-null int64
year                  45681

,artist_familiarity,artist_hotttnesss,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,release,song_hotttnesss,song_id,title,duration,end_of_fade_in,key,loudness,mode,start_of_fade_out,tempo,time_signature,year
0,0.649822,0.394032,ARYZTJS1187B98C555,NaN,NaN,NaN,Faster Pussy cat,Monster Ballads X-Mas,0.542899,SOQMMHC12AB0180CB8,Silent Night,252.05506,2.049,10,-4.829,0,236.635,87.002,4,2003.0
1,0.439604,0.356992,ARMVN3U1187FB3A1EB,NaN,NaN,NaN,Karkkiautomaatti,Karkuteill\xc3\xa4,0.299877,SOVFVAK12A8C1350D9,Tanssi vaan,156.55138,0.258,9,-10.555,1,148.660,150.778,1,1995.0
2,0.643681,0.437504,ARGEKB01187FB50750,55.8578,"Glasgow, Scotland",-4.24251,Hudson Mohawke,Butter,0.617871,SOGTUKN12AB017F4F1,No One Could Ever,138.97098,0.000,7,-2.060,1,138.971,177.768,4,2006.0
3,0.448501,0.372349,ARNWYLR1187B9B2F9C,NaN,NaN,NaN,Yerba Brava,De Culo,NaN,SOBNYVR12A8C13558C,Si Vos Quer\xc3\xa9s,145.05751,0.000,7,-4.654,1,138.687,87.433,4,2003.0
4,0.000000,0.000000,AREQDTE1269FB37231,NaN,NaN,NaN,Der Mystic,Rene Ablaze Presents Winter Sessions,NaN,SOHSBXH12A8C13B0DF,Tangle Of Aspens,514.29832,0.000,5,-7.806,0,506.717,140.035,4,NaN


## Upload User Data

In [4]:
users_path = ('/Users/ivettetapia 1/Symbolic Link Seagate Drive/Springboard/Capstone 1_Mus_Recomend/Data/msd_user_data.csv')

users = pd.read_csv(users_path, na_values = 'NaN', index_col = 0)

print(users.info())

users.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48373586 entries, 0 to 48373585
Data columns (total 4 columns):
user_id       object
song_id       object
play_count    int64
play          int64
dtypes: int64(2), object(2)
memory usage: 1.8+ GB
None


,user_id,song_id,play_count,play
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1,1
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAPDEY12A81C210A9,1,1
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2,1
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFNSP12AF72A0E22,1,1
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFOVM12A58A7D494,1,1


<font color='green'> **My machine cannot handle all the user data. Set thresholds of user song listens. The goal of this is to have a principled way of eliminating users.**</font>

In this case, I am eliminating users with less than < 45 song listens and > 452. As shown in the exploratory analysis, there is a big long tail that begins at ~450 song listens. Moreover, unique song listens beyond 2,000 are suspect.

Users with less than 90 listens have an extremely sparse data.

In [5]:
thres_1 = len(songs)*0.00005

thres_2 = len(songs)*0.0005

print("Less than threshold of unique song plays:", thres_1)
print('More than threshold of unique song plays:', thres_2)

('Less than threshold of unique song plays:', 45.2856)
('More than threshold of unique song plays:', 452.856)


<font color='green'> **Group users by user_id and sum**  </font>

In [6]:
users_grouped = users.groupby('user_id',as_index=False).sum()

print('Total Users: ', len(users_grouped))

print(users_grouped.head())

('Total Users: ', 1019318)
                                    user_id  play_count  play
0  00000b722001882066dff9d2da8a775658053ea0          13    12
1  00001638d6189236866af9bbf309ae6c2347ffdc          56    14
2  0000175652312d12576d9e6b84f600caa24c4715          19    12
3  00001cf0dce3fb22b0df0f3a1d9cd21e38385372          25    18
4  0000267bde1b3a70ea75cf2b2d216cb828e3202b          27    25


In [7]:
users_grouped_filt = users_grouped[(users_grouped['play'] > thres_1) & 
                                   (users_grouped['play'] < thres_2)]

print('Filtered Users (i.e. remaining users): ', len(users_grouped_filt)) 

#Eliminated around 2/3 of users using threshold criteria

('Filtered Users (i.e. remaining users): ', 313689)


<font color='green'> **Filter Users DF by the filtered users list** </font>

In [8]:
users_filt = users.merge(users_grouped_filt[['user_id']], on = 'user_id', how='inner',suffixes=('_x', '_y'))

In [9]:
users_filt.info()

users_filt.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31754801 entries, 0 to 31754800
Data columns (total 4 columns):
user_id       object
song_id       object
play_count    int64
play          int64
dtypes: int64(2), object(2)
memory usage: 1.2+ GB


,user_id,song_id,play_count,play
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1,1
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAPDEY12A81C210A9,1,1
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2,1
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFNSP12AF72A0E22,1,1
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFOVM12A58A7D494,1,1


In [10]:
#Sanity check. Confirm the number of unique users is the same.

users_filt.user_id.nunique()

313689

## Upload Artist Tags Data

In [11]:
artist_tags_path = ('/Users/ivettetapia 1/Symbolic Link Seagate Drive/Springboard/Capstone 1_Mus_Recomend/Data/msd_tags.csv')

tags = pd.read_csv(artist_tags_path, index_col = 0, na_values = 'NaN',
                        dtype = {0:'int', 1:'str',2:'str',3:'str'})

print(tags.info())

tags.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1621124 entries, 0 to 1621123
Data columns (total 3 columns):
artist_id    1621124 non-null object
mbtag        798145 non-null object
terms        1621118 non-null object
dtypes: object(3)
memory usage: 49.5+ MB
None


,artist_id,mbtag,terms
0,AR002UA1187B9A637D,uk,garage rock
1,AR002UA1187B9A637D,uk,country rock
2,AR002UA1187B9A637D,uk,free jazz
3,AR002UA1187B9A637D,uk,oi
4,AR002UA1187B9A637D,uk,space rock


## Convert objects fields

<font color='green'> **Convert objects to string and floats to int. This is to facilitate conversion to GraphLab's SFrame format. An SF will not accept a pandas column whose type is an object.**</font>

In [12]:
#Objects in the user df

users_filt[['user_id','song_id']] = users_filt[['user_id','song_id']].astype('|S250')

In [13]:
#Objects in the songs df

songs[['artist_id','artist_location', 'artist_name','release','song_id','title']] = songs[['artist_id','artist_location', 'artist_name','release','song_id','title']].astype('|S250')

In [14]:
#Objects in the tags df

tags[['artist_id','mbtag','terms']] = tags[['artist_id','mbtag','terms']].astype('|S250')

## Convert Pandas DataFrames to SFrames

<font color='green'> **Convert User Data DF to GraphLab dataframe format to use GraphLab libraries recommendation algorithms.**</font>

In [15]:
users_sf = gl.SFrame(data=users_filt)

This non-commercial license of GraphLab Create for academic use is assigned to ivette1989@gmail.com and will expire on August 19, 2019.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1535473521.log


In [16]:
#Randomly shuffle the data. This is a preprocesing step.

random_seed=42 #this random seed will be the same throughout.

users_sf = gl.cross_validation.shuffle(users_sf,random_seed = random_seed)

users_sf

user_id,song_id,play_count,play
d435af9885e7a99a8fc330b7e4693e8efd24932f ...,SOLSFPA12A6D4FAFB2,1,1
c605e48b580ed998a999005675f6bc65496dc437 ...,SOGGTYQ12A8C133BAD,1,1
850452f59960157c712fd1ec41894d2f632d13e4 ...,SOKKKZW12A6D4F7F88,2,1
c3ef1d45ada29646c9031c327fd02566dd5a732c ...,SOQIUXU12A67ADAEF4,2,1
be963f98fac8af2cbc4760e085ad1b3febea7d56 ...,SOCVNLL12A8C13B4EA,1,1
09e4e9b9ba6dd8aa8a5f1c3b58e57bacfc14090d ...,SOLIXJQ12A6D4F8C4D,1,1
e831eb92afb6fc49ce77067a5c5b3717cf5e7d1e ...,SOYYJMW12A6D4F8636,10,1
46d82b8750edb1fd9f6d4f3b0238f56030f478c2 ...,SOHRQFL12A6D4F3C8A,1,1
f1b57127863836c14413d4bfd9f4bd9827a82d19 ...,SOVMGAJ12AB0181580,1,1
c9e30e24637e4cf6f0a35ed4ebacf6f9e1597e17 ...,SOQSCOQ12A81C234C8,1,1


<font color='green'> **Convert Song Data DF to GraphLab dataframe format to use GraphLab libraries recommendation algorithms.**</font>

In [17]:
songs_sf = gl.SFrame(data=songs)

In [18]:
#Randomly shuffle the data.This is a preprocesing step.

songs_sf = gl.cross_validation.shuffle(songs_sf,random_seed=random_seed)

songs_sf

artist_familiarity,artist_hotttnesss,artist_id,artist_latitude,artist_location,artist_longitude
0.676279034535,0.440250907534,ARHAIBP1187B99FB99,40.65507,"Brooklyn, NY",-73.94888
0.609602860049,0.424077070114,ARNIDSZ1187FB564B5,nan,Germany/Munich,nan
0.556038044172,0.440037146261,AR5OFO01187B98C9B1,nan,nan,nan
0.589357755432,0.46550988432,ARNW1JU1187B9B88D3,30.9742,Louisiana,-91.52382
0.655429099673,0.445779051107,ARMSWRC1187FB4CD02,nan,"Reston, Virginia",nan
0.512814937369,0.353487741638,ARFSC4Y1187FB459C1,nan,nan,nan
0.650177336245,0.40496324532,AR2B3K21187B9B9A2E,nan,nan,nan
0.691125976977,0.557643463149,AR3W2UH1187FB4D57C,51.16418,Germany,10.45415
0.636119207143,0.363499577136,ARL6UP41187FB5C604,nan,nan,nan
0.624668645159,0.409180608729,ARG7XSH1187FB49058,nan,nan,nan


<font color='green'> **Convert Artist Tags Data DF to GraphLab dataframe format to use GraphLab libraries recommendation algorithms.**</font>

In [19]:
tags_sf = gl.SFrame(data=tags)

In [20]:
#Randomly shuffle the data.This is a preprocesing step.

tags_sf = gl.cross_validation.shuffle(tags_sf,random_seed=random_seed)

tags_sf

artist_id,mbtag,terms
AR8S3TE1187B99B4EF,nan,jump up
ARRXFTU1187B9AD2C1,nan,singer-songwriter
ARVILQ41187FB3B1D8,rock,trance
ARUR3Q71187FB594BA,german,ukraine
AREMTIU120AB95F613,male vocalists,aor
ARYAOT71187B9AA2A0,nan,jazz
ARLPWP41187FB5AA33,nan,alternative rock
ARQM3CJ1187B9B7AD1,neighbours,dance pop
ARSJOD21187B998EA6,crossover thrash,uk hardcore
AR4H6UV1187B9A7BFA,nettetal,big beat


## Create Cross-Validation Folds & User Data Train Test Split 

In [21]:
#Create five folds in user data

folds = gl.cross_validation.KFold(users_sf, 5)

Create a recommender-friendly train-test split of the provided data set.

The test dataset is generated by first choosing max_num_users out of the total number of users in dataset. Then, for each of the chosen test users, a portion of the user’s items (determined by item_test_proportion) is randomly chosen to be included in the test set. This split allows the training data to retain enough information about the users in the testset, so that adequate recommendations can be made. The total number of users in the test set may be fewer than max_num_users if a user was chosen for the test set but none of their items are selected.

In [22]:
# Create test and train splits based on user_ids

max_num_users= round(len(users_grouped_filt)*.3) #max number of users to be included in the test set

users_train, users_test = gl.recommender.util.random_split_by_user(users_sf,
                                                                   user_id = 'user_id',
                                                                   item_id = 'song_id',
                                                                   item_test_proportion = 0.3,
                                                                   max_num_users= max_num_users,
                                                                   random_seed=random_seed)

print('Length to Train and Test Samples:', len(users_train),len(users_test))

('Length to Train and Test Samples:', 28899739, 2855062)


## Create Baseline: Song Popularity Based Recommender

The Popularity Model ranks an item according to its overall popularity.

This model is used as a frame of reference for the subsequent approaches. This recomender does  not have any personalization at all. It recommends the most popular items to users. **There are no hyperparameters to set.**

When making recommendations, the items are scored by the number of times it is seen in the training set. The item scores are the same for all users. Hence the recommendations are not tailored for individuals.

The Popularity Recommender is simple and fast and provides a reasonable baseline. It can work well when observation data is sparse. It can be used as a “background” model for new users.

Model Documentation: https://turi.com/products/create/docs/generated/graphlab.recommender.popularity_recommender.create.html#graphlab.recommender.popularity_recommender.create

<font color='green'> **Create Song Popularity Recomender**</font>

In [23]:
song_popularity =  gl.popularity_recommender.create(observation_data = users_train, 
                                                    item_id = 'song_id',
                                                    user_id='user_id',
                                                    item_data = songs_sf,
                                                    random_seed=random_seed,
                                                    verbose=False)

Recsys training: model = popularity

<font color='green'> **Save Trained Model**</font>

In [24]:
song_popularity.save('song_popularity_trained')

<font color='green'> **Model Summary**</font>

In [25]:
song_popularity.summary()

Class                            : PopularityRecommender

Schema
------
User ID                          : user_id
Item ID                          : song_id
Target                           : None
Additional observation features  : 0
User side features               : []
Item side features               : ['artist_familiarity', 'artist_hotttnesss', 'artist_id', 'artist_latitude', 'artist_location', 'artist_longitude', 'artist_name', 'release', 'song_hotttnesss', 'song_id', 'title', 'duration', 'end_of_fade_in', 'key', 'loudness', 'mode', 'start_of_fade_out', 'tempo', 'time_signature', 'year']

Statistics
----------
Number of observations           : 28899739
Number of users                  : 313689
Number of items                  : 929322

Training summary
----------------
Training time                    : 0.2449

Model Parameters
----------------
Model class                      : PopularityRecommender



In [26]:
song_popularity.list_fields()

['data_load_time',
 'item_id',
 'item_predictions',
 'item_side_data_column_names',
 'item_side_data_column_types',
 'model_name',
 'num_features',
 'num_item_side_features',
 'num_items',
 'num_observations',
 'num_user_side_features',
 'num_users',
 'observation_data_column_names',
 'random_seed',
 'target',
 'training_time',
 'user_id',
 'user_side_data_column_names',
 'user_side_data_column_types']

<font color='green'> **Load Saved Song Popularity Recomender**</font>

In [27]:
loaded_song_pop = gl.load_model('song_popularity_trained')

<font color='green'> **Evaluate Song Popularity Recomender Precision and Recall on Test Data Using Train-Split**</font>

In [28]:
eval_song_pop = loaded_song_pop.evaluate_precision_recall(users_test)

recommendations finished on 1000/94107 queries. users per second: 95.0635

recommendations finished on 2000/94107 queries. users per second: 93.7064

recommendations finished on 3000/94107 queries. users per second: 94.5824

recommendations finished on 4000/94107 queries. users per second: 96.4263

recommendations finished on 5000/94107 queries. users per second: 98.609

...

recommendations finished on 90000/94107 queries. users per second: 119.138

recommendations finished on 91000/94107 queries. users per second: 119.308

recommendations finished on 92000/94107 queries. users per second: 119.195

recommendations finished on 93000/94107 queries. users per second: 119.137

recommendations finished on 94000/94107 queries. users per second: 119.189

## Prepare Data for Artist Popularity Based Recomender

In [29]:
#Join user listen dataset with artist_id. Keep songs that have artist_id matches on them.

users_art_sf = users_sf.join(songs_sf['song_id', 'artist_id'], on = 'song_id', how='inner')

In [30]:
#Create five folds in user data that includes artist.

folds_art = gl.cross_validation.KFold(users_art_sf, 5)

In [31]:
train_art, test_art = gl.recommender.util.random_split_by_user(users_art_sf,
                                                               user_id = 'user_id',
                                                               item_id = 'song_id',
                                                               item_test_proportion = 0.3,
                                                               max_num_users= 90000,
                                                               random_seed=random_seed)

print('Length of train and test datasets:', len(train_art), len(test_art))

('Length of train and test datasets:', 24378213, 2299014)


## Create Baseline: Artist Popularity Based Recommender

The Popularity Model ranks an item according to its overall popularity.

This model is used as a frame of reference for the subsequent approaches. This recomender does  not have any personalization at all. It recommends the most popular artists to users. **There are no hyperparameters to set.**

When making recommendations, the items are scored by the number of times it is seen in the training set. The item scores are the same for all users. Hence the recommendations are not tailored for individuals.

The Popularity Recommender is simple and fast and provides a reasonable baseline. It can work well when observation data is sparse. It can be used as a “background” model for new users.

Model Documentation: https://turi.com/products/create/docs/generated/graphlab.recommender.popularity_recommender.create.html#graphlab.recommender.popularity_recommender.create

<font color='green'>**Create Artist Popularity Recomender** </font>

In [32]:
artist_popularity = gl.popularity_recommender.create(observation_data = train_art['artist_id','user_id'], 
                                                    item_id = 'artist_id',
                                                    user_id='user_id',
                                                    item_data = songs_sf,
                                                    random_seed=random_seed,
                                                    verbose=False)

Recsys training: model = popularity

<font color='green'> **Save Trained Model**</font>

In [33]:
artist_popularity.save('artist_popularity_trained')

<font color='green'>**Model Summary** </font>

In [34]:
artist_popularity.summary()

Class                            : PopularityRecommender

Schema
------
User ID                          : user_id
Item ID                          : artist_id
Target                           : None
Additional observation features  : 0
User side features               : []
Item side features               : ['artist_familiarity', 'artist_hotttnesss', 'artist_id', 'artist_latitude', 'artist_location', 'artist_longitude', 'artist_name', 'release', 'song_hotttnesss', 'song_id', 'title', 'duration', 'end_of_fade_in', 'key', 'loudness', 'mode', 'start_of_fade_out', 'tempo', 'time_signature', 'year']

Statistics
----------
Number of observations           : 24378213
Number of users                  : 313689
Number of items                  : 44421

Training summary
----------------
Training time                    : 0.0118

Model Parameters
----------------
Model class                      : PopularityRecommender



In [35]:
artist_popularity.list_fields()

['data_load_time',
 'item_id',
 'item_predictions',
 'item_side_data_column_names',
 'item_side_data_column_types',
 'model_name',
 'num_features',
 'num_item_side_features',
 'num_items',
 'num_observations',
 'num_user_side_features',
 'num_users',
 'observation_data_column_names',
 'random_seed',
 'target',
 'training_time',
 'user_id',
 'user_side_data_column_names',
 'user_side_data_column_types']

<font color='green'> **Load Saved Artist Popularity Recomender**</font>

In [36]:
loaded_artist_pop = gl.load_model('artist_popularity_trained')

<font color='green'>**Evaluate Artist Popularity Recomender Precision and Recall Using Test-Split** </font>

In [37]:
eval_artist_pop = loaded_artist_pop.evaluate_precision_recall(test_art)

recommendations finished on 1000/90000 queries. users per second: 486.024

recommendations finished on 2000/90000 queries. users per second: 679.255

recommendations finished on 3000/90000 queries. users per second: 801.207

recommendations finished on 4000/90000 queries. users per second: 857.442

recommendations finished on 5000/90000 queries. users per second: 932.499

...

recommendations finished on 86000/90000 queries. users per second: 1264.86

recommendations finished on 87000/90000 queries. users per second: 1265.03

recommendations finished on 88000/90000 queries. users per second: 1263.47

recommendations finished on 89000/90000 queries. users per second: 1265.37

recommendations finished on 90000/90000 queries. users per second: 1264.67

## Content - Based Recommender

Create a content-based recommender model in which the similarity between the items recommended is determined by the content of those items rather than learned from user interaction data.

The similarity score between two items is calculated by first computing the similarity between the item data for each column, then taking a weighted average of the per-column similarities to get the final similarity. The recommendations are generated according to the average similarity of a candidate item to all the items in a user’s set of rated items.

Model Documentation:

https://turi.com/products/create/docs/generated/graphlab.recommender.item_content_recommender.create.html#graphlab.recommender.item_content_recommender.create

In [38]:
content = gl.item_content_recommender.create(item_data = songs_sf,
                                             item_id = 'song_id',
                                             observation_data = users_train,
                                             user_id = 'user_id')

('Applying transform:\n', Class             : AutoVectorizer

Model Fields
------------
Features          : ['artist_familiarity', 'artist_hotttnesss', 'artist_id', 'artist_latitude', 'artist_location', 'artist_longitude', 'artist_name', 'release', 'song_hotttnesss', 'title', 'duration', 'end_of_fade_in', 'key', 'loudness', 'mode', 'start_of_fade_out', 'tempo', 'time_signature', 'year']
Excluded Features : ['song_id']

Column              Type   Interpretation  Transforms                         Output Type
------------------  -----  --------------  ---------------------------------  -----------
artist_familiarity  float  numerical       None                               float      
artist_hotttnesss   float  numerical       None                               float      
artist_id           str    categorical     None                               str        
artist_latitude     float  numerical       None                               float      
artist_location     str    categorica

Recsys training: model = item_content_recommender

Defaulting to brute force instead of ball tree because there are multiple distance components.


Starting brute force nearest neighbors model training.

Starting pairwise querying.

...

| 6574         | 6e+09   | 0.725921    | 1h 0m        |

| 6576         | 6e+09   | 0.7261      | 1h 0m        |

| 6577         | 6e+09   | 0.726259    | 1h 0m        |

[ERROR] graphlab.toolkits._main: Toolkit error: Toolkit cancelled by user.


ToolkitError: Toolkit cancelled by user.

<font color='green'> **Save Trained Model**</font>

In [40]:
content.save('content_trained')

NameError: name 'content' is not defined

<font color='green'>**Model Summary** </font>

In [ ]:
content.summary()

In [ ]:
content.list_fields()

<font color='green'> **Load Saved Artist Popularity Recomender**</font>

In [ ]:
loaded_content = gl.load_model('content_trained')

<font color='green'>**Evaluate Artist Popularity Recomender Precision and Recall Using Test-Split** </font>

In [ ]:
eval_content = content.evaluate_precision_recall(users_test)

## Item - Item Recommender

Create a recommender that uses item-item similarities based on users in common.

Documentation for this model:

https://turi.com/products/create/docs/generated/graphlab.recommender.item_similarity_recommender.create.html#graphlab.recommender.item_similarity_recommender.create

<font color='green'>**Create Item - Item Recomender** </font>

In [39]:
item_rec = gl.item_similarity_recommender.create(observation_data = users_train,
                                                 user_id = 'user_id',
                                                 item_id = 'song_id',
                                                 item_data = songs_sf,
                                                 similarity_type = 'jaccard')

Recsys training: model = item_similarity

Warning: Ignoring columns play_count, play;

To use one of these as a target column, set target = <column_name>

and use a method that allows the use of a target.

Preparing data set.

...

| 42m 39s                             | 100              | 929322          |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 2567.43s

<font color='green'> **Save Trained Model**</font>

In [41]:
item_rec.save('item_rec_trained')

<font color='green'>**Model Summary** </font>

In [42]:
item_rec.summary()

Class                            : ItemSimilarityRecommender

Schema
------
User ID                          : user_id
Item ID                          : song_id
Target                           : None
Additional observation features  : 0
User side features               : []
Item side features               : ['artist_familiarity', 'artist_hotttnesss', 'artist_id', 'artist_latitude', 'artist_location', 'artist_longitude', 'artist_name', 'release', 'song_hotttnesss', 'song_id', 'title', 'duration', 'end_of_fade_in', 'key', 'loudness', 'mode', 'start_of_fade_out', 'tempo', 'time_signature', 'year']

Statistics
----------
Number of observations           : 28899739
Number of users                  : 313689
Number of items                  : 929322

Training summary
----------------
Training time                    : 2567.4373

Model Parameters
----------------
Model class                      : ItemSimilarityRecommender
threshold                        : 0.001
similarity_type            

In [43]:
item_rec.list_fields()

['data_load_time',
 'degree_approximation_threshold',
 'item_id',
 'item_side_data_column_names',
 'item_side_data_column_types',
 'max_data_passes',
 'max_item_neighborhood_size',
 'model_name',
 'nearest_neighbors_interaction_proportion_threshold',
 'num_features',
 'num_item_side_features',
 'num_items',
 'num_observations',
 'num_user_side_features',
 'num_users',
 'observation_data_column_names',
 'seed_item_set_size',
 'similarity_type',
 'sparse_density_estimation_sample_size',
 'target',
 'target_memory_usage',
 'threshold',
 'training_method',
 'training_rmse',
 'training_time',
 'user_id',
 'user_side_data_column_names',
 'user_side_data_column_types']

<font color='green'> **Load Saved Artist Popularity Recomender**</font>

In [44]:
loaded_item = gl.load_model('item_rec_trained')

<font color='green'>**Evaluate Item-Based Recomender Precision and Recall Using Test-Split** </font>

In [46]:
eval_item = loaded_item.evaluate_precision_recall(users_test)

recommendations finished on 1000/94107 queries. users per second: 136.621

recommendations finished on 2000/94107 queries. users per second: 101.265

recommendations finished on 3000/94107 queries. users per second: 94.8636

recommendations finished on 4000/94107 queries. users per second: 103.099

recommendations finished on 5000/94107 queries. users per second: 108.522

...

recommendations finished on 90000/94107 queries. users per second: 147.17

recommendations finished on 91000/94107 queries. users per second: 147.232

recommendations finished on 92000/94107 queries. users per second: 147.29

recommendations finished on 93000/94107 queries. users per second: 147.34

recommendations finished on 94000/94107 queries. users per second: 147.387

## Ranking Matrix Factorization Recomender

Matrix factorization algorithm for implicit data. Determine n-ranked items since predicting 'ratings' does not makes sense for implicit of data.

Create a RankingFactorizationRecommender that learns latent factors for each user and item and uses them to make rating predictions.

A RankingFactorizationRecommender learns latent factors for each user and item and uses them to rank recommended items according to the likelihood of observing those (user, item) pairs. This is commonly desired when performing collaborative filtering for implicit feedback datasets or datasets with explicit ratings for which ranking prediction is desired.

Model documentation:

https://turi.com/products/create/docs/generated/graphlab.recommender.ranking_factorization_recommender.create.html#graphlab.recommender.ranking_factorization_recommender.create

<font color='green'>**Create Recomender** </font>

In [49]:
matfac_rec1 = gl.ranking_factorization_recommender.create(users_train,
                                                          user_id = 'user_id',
                                                          item_id = 'song_id',
                                                          regularization = 1e-9,
                                                          linear_regularization = 1e-9,
                                                          solver = 'ials')

Recsys training: model = ranking_factorization_recommender

Preparing data set.

Data has 28899739 observations with 313689 users and 367021 items.

Data prepared in: 86.6228s

Training ranking_factorization_recommender for recommendations.

...

| 23      | 15m 39s      | 0.921904                  |

| 24      | 16m 15s      | 0.921898                  |

| FINAL   | 16m 15s      | 0.921898                  |

+---------+--------------+---------------------------+

Optimization Complete: Iteration limit reached.

<font color='green'> **Save Trained Models**</font>

In [ ]:
matfac_rec1.save('matfac_rec_trained1')

In [ ]:
matfac_rec2.save('matfac_rec_trained2')

<font color='green'>**MF Model 1 Summary** </font>

In [ ]:
matfac_rec.summary()

In [ ]:
matfac_rec.list_fields()

<font color='green'>**MF Model 2 Summary** </font>

In [ ]:
matfac_rec2.summary()

In [ ]:
matfac_rec2.list_fields()

<font color='green'> **Load Saved Matrix Factorization Artist Popularity Recomenders**</font>

In [ ]:
loaded_matfac_rec1 = gl.load_model('matfac_rec_trained1')

loaded_matfac_rec2 = gl.load_model('matfac_rec_trained')

<font color='green'>**Evaluate Item-Based Recomender Precision and Recall Using Test-Split** </font>

In [ ]:
eval_matfact1 = loaded_matfac_rec1.evaluate_precision_recall(users_test)

In [ ]:
eval_matfact2 = loaded_matfac_rec2.evaluate_precision_recall(users_test)

## Model Evaluation Using Train - Test Split

<font color='green'>**Explore how to get evaluation results out.** </font>

In [ ]:
print(type(eval_song_pop))

eval_song_pop.keys()

<font color='green'>**Create Columns** </font>

In [ ]:
cutoffs = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 16, 21, 26, 31, 36, 41, 46]

c_songpop = gl.SArray(['Song Popularity'] * len(cutoffs))
c_artpop = gl.SArray(['Artist Popularity'] * len(cutoffs))
c_cont = gl.SArray(['Content-Based'] * len(cutoffs))
c_item = gl.SArray(['Item-Item'] * len(cutoffs))
c_matfact1 = gl.SArray(['Matrix Factorization (SVD) 1'] * len(cutoffs))
c_matfact2 = gl.SArray(['Matrix Factorization (SVD) 2'] * len(cutoffs))

<font color='green'>**Add Identifying Columns & Convert to DataFrame** </font>

In [ ]:
pres_recall1 = eval_song_pop['precision_recall_overall'].add_column(c_songpop, name='Recommender Type').to_dataframe()

pres_recall2 = eval_artist_pop['precision_recall_overall'].add_column(c_artpop, name='Recommender Type').to_dataframe()

pres_recall3 = eval_content['precision_recall_overall'].add_column(c_cont, name='Recommender Type').to_dataframe()

pres_recall4 = eval_item['precision_recall_overall'].add_column(c_item, name='Recommender Type').to_dataframe()

pres_recall5 = eval_matfact1['precision_recall_overall'].add_column(c_matfact, name='Recommender Type').to_dataframe()

pres_recall6 = eval_matfact1['precision_recall_overall'].add_column(c_matfact, name='Recommender Type').to_dataframe()

<font color='green'>**Concatenate Frames** </font>

In [ ]:
frames = [pres_recall1, pres_recall2, pres_recall3, pres_recall4, pres_recall5, pres_recall6]

precision_recall_overall = pd.concat(frames)

precision_recall_overall.head()

<font color='green'>**Add a column with F-score** </font>

In [ ]:
precision_recall_overall['FScore'] = 2*((precision_recall_overall['precision'] * precision_recall_overall['recall'])/
                                        (precision_recall_overall['precision'] + precision_recall_overall['recall']))


<font color='green'>**Add a column with True Positives ** </font>

In [ ]:
#precision_recall_overall['True_Positives'] = precision_recall_overall['precision']*
                                             #precision_recall_overall['index']*

<font color='green'>**Add a column with False Positives ** </font>



In [ ]:
#precision_recall_overall['True_Positives'] = precision_recall_overall['precision']*
                                             #precision_recall_overall['index']*

## Graph Evaluation Results - *Test Split (70-30)*

In [ ]:
%matplotlib inline

sns.set_style("whitegrid")

sns.despine()

sns.lineplot(x='recall', y='precision', data = precision_recall_overall, 
             hue='Recommender Type', markers=True)

plt.title('Precision-Recall Comparisons \n')

plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

#plt.xticks(np.arange(0, 1.2, step=0.2))
#plt.yticks(np.arange(0, 1.2, step=0.2))

plt.show()

In [ ]:
dill.dump_session('notebook_env.db')

## IDEAS & NEXT STEPS

+ Add other accuracy metrics
+ Other options  - content based/clutering/esemble. 
+ Limitations

# End of Notebook